# Windowing and Framing

In [1]:
import whisper
whisper.available_models()

['tiny.en',
 'tiny',
 'base.en',
 'base',
 'small.en',
 'small',
 'medium.en',
 'medium',
 'large-v1',
 'large-v2',
 'large-v3',
 'large']

In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('..', '..')) # or the path to your source code
sys.path.insert(0, module_path)
from src.data_loading import load_iemocap, load_meld, load_emodb, load_ravdess, load_tess, load_crema_d, filter_emotions

iemocap = filter_emotions(load_iemocap())
meld = filter_emotions(load_meld())
meld_train = filter_emotions(load_meld(split='train'))
emodb = filter_emotions(load_emodb())
ravdess = filter_emotions(load_ravdess())
tess = filter_emotions(load_tess())
crema_d = filter_emotions(load_crema_d())

/Users/krazmic/Documents/GitHub/Repos/EmoReA/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Subfolders in the dataset: ['IEMOCAP_full_release']
Subfolders in the dataset: ['JSON files', 'MELD.Raw']
Subfolders in the raw data: ['dia47_utt11.mp4', 'dia35_utt2.mp4', 'dia47_utt9.mp4', 'dia167_utt14.mp4', 'dia232_utt4.mp4', '._dia118_utt11.mp4', 'dia34_utt2.mp4', 'dia268_utt8.mp4', 'dia233_utt4.mp4', '._dia39_utt1.mp4', '._dia230_utt3.mp4', 'dia137_utt0.mp4', '._dia268_utt11.mp4', '._dia231_utt3.mp4', 'dia136_utt0.mp4', '._dia38_utt1.mp4', '._dia34_utt16.mp4', '._dia128_utt2.mp4', 'dia28_utt7.mp4', 'dia195_utt8.mp4', 'dia22_utt15.mp4', '._dia129_utt2.mp4', '._dia100_utt5.mp4', 'dia29_utt7.mp4', 'dia103_utt2.mp4', 'final_videos_testdia48_utt3.mp4', 'dia154_utt12.mp4', '._dia95_utt11.mp4', '._dia205_utt1.mp4', 'dia102_utt2.mp4', 'dia107_utt8.mp4', 'dia253_utt11.mp4', '._dia52_utt5.mp4', 'dia175_utt3.mp4', 'dia93_utt0.mp4', '._dia272_utt0.mp4', '._dia192_utt5.mp4', 'dia174_utt3.mp4', '._dia53_utt5.mp4', 'dia175_utt13.mp4', 'dia92_utt0.mp4', 'dia191_utt2.mp4', 'dia77_utt1.mp4', 'dia25

In [4]:
from line_profiler import LineProfiler

lp = LineProfiler()
lp.add_function(FileProcessor._process_audio)
lp_wrapper = lp(processor.process_file)

# Run a few files, not the whole dataset (too slow)
for f in meld['filename'][:3]:
    lp_wrapper(f)

lp.print_stats()


Timer unit: 1e-09 s

Total time: 2.73293 s
File: /Users/krazmic/Documents/GitHub/Repos/EmoReA/emorea-backend/src/processor.py
Function: FileProcessor.process_file at line 23

Line #      Hits         Time  Per Hit   % Time  Line Contents
    23                                               def process_file(self, file_path):
    24                                                   """Main processing method"""
    25         3     180000.0  60000.0      0.0          if not os.path.isfile(file_path):
    26                                                       return {"error": "File not found"}
    27                                           
    28         3      63000.0  21000.0      0.0          file_type = self._detect_file_type(file_path)
    29                                           
    30         3          0.0      0.0      0.0          try:
    31         3       1000.0    333.3      0.0              if file_type == 'text':
    32                                             

In [2]:
from src.processor import FileProcessor
from src.recognizers import SpeechEmotionRecognizer
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

processor = FileProcessor()
meld['info'] = meld['filename'].progress_apply(processor.process_file)

  5%|▍         | 123/2610 [02:01<41:04,  1.01it/s] 


KeyboardInterrupt: 

In [6]:
meld['text'] = meld['info'].progress_apply(lambda x: x['text'])
meld

100%|██████████| 2610/2610 [00:00<00:00, 253107.06it/s]

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,filename,label,info,text
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378",/Users/krazmic/.cache/kagglehub/datasets/bhand...,surprise,"{'text': ' Little R кв from Knox, pounding and...","Little R кв from Knox, pounding and bushing u..."
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385",/Users/krazmic/.cache/kagglehub/datasets/bhand...,angry,{'text': ' Uh... It's a moment of computer wor...,Uh... It's a moment of computer work that won...
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520",/Users/krazmic/.cache/kagglehub/datasets/bhand...,neutral,"{'text': '', 'segments': [], 'audio': {'raw': ...",
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146",/Users/krazmic/.cache/kagglehub/datasets/bhand...,neutral,"{'text': ' А то ведь какая-то животных.', 'seg...",А то ведь какая-то животных.
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833",/Users/krazmic/.cache/kagglehub/datasets/bhand...,happy,"{'text': ' Oh, yeah. Yeah.', 'segments': [{'st...","Oh, yeah. Yeah."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2605,2760,"Yeah, I mean, come on Ross, no one will even n...",Rachel,neutral,neutral,279,11,6,4,"00:14:35,457","00:14:40,211",/Users/krazmic/.cache/kagglehub/datasets/bhand...,neutral,{'text': ' but a hundred years younger Army av...,but a hundred years younger Army average inte...
2606,2761,They’re not listening too me?,Ross,surprise,negative,279,12,6,4,"00:14:42,256","00:14:43,840",/Users/krazmic/.cache/kagglehub/datasets/bhand...,surprise,"{'text': '1', 'segments': [{'start': 0.0, 'end...",1
2607,2762,Of course they’re listening to you! Everybody ...,Rachel,neutral,neutral,279,13,6,4,"00:14:44,008","00:14:48,511",/Users/krazmic/.cache/kagglehub/datasets/bhand...,neutral,"{'text': ' Of course not, lesson and child. A ...","Of course not, lesson and child. A vår correc..."
2608,2763,Monica you really think I should try this phas...,Ross,neutral,neutral,279,14,6,4,"00:14:48,138","00:14:52,390",/Users/krazmic/.cache/kagglehub/datasets/bhand...,neutral,"{'text': ' you', 'segments': [{'start': 0.0, '...",you


In [7]:
meld.to_csv("MELD_transcripts-32kHz.csv")

In [3]:
ser = SpeechEmotionRecognizer("logreg_ERTCM.joblib")

In [4]:
raw_audios = data.progress_apply(lambda x: (x['audio']['raw'], x['audio']['sample_rate']))
preds = raw_audios.progress_apply(lambda x: ser.analyze(*x))

100%|██████████| 100/100 [00:06<00:00, 14.88it/s]


In [5]:
preds

2742    {'emotions': ['neutral']}
4314       {'emotions': ['fear']}
1838        {'emotions': ['sad']}
2875        {'emotions': ['sad']}
482         {'emotions': ['sad']}
                  ...            
1307    {'emotions': ['neutral']}
4108      {'emotions': ['angry']}
3570    {'emotions': ['neutral']}
3401    {'emotions': ['neutral']}
485     {'emotions': ['neutral']}
Name: filename, Length: 100, dtype: object

In [ ]:
iemocap_feats = pd.read_csv("IEMOCAP_GeMAPS.csv")
emobd_feats = pd.read_csv("EMODB_GeMAPS.csv")
ravdess_feats = pd.read_csv("RAVDESS_GeMAPS.csv")
tess_feats = pd.read_csv("TESS_GeMAPS.csv")
crema_d_feats = pd.read_csv("CREMA-D_GeMAPS.csv")
meld_feats = pd.read_csv("MELD_GeMAPS.csv")
meld_train_feats = pd.read_csv("MELD_train_GeMAPS.csv")

dict_keys(['id', 'seek', 'start', 'end', 'text', 'tokens', 'temperature', 'avg_logprob', 'compression_ratio', 'no_speech_prob'])

In [8]:
ex['segments'][1]

{'id': 1,
 'seek': 0,
 'start': 8.28,
 'end': 11.540000000000001,
 'text': ' Minded little vampire and I hope to God. I never see you again',
 'tokens': [50778,
  13719,
  292,
  707,
  28592,
  293,
  286,
  1454,
  281,
  1265,
  13,
  286,
  1128,
  536,
  291,
  797,
  50941],
 'temperature': 0.0,
 'avg_logprob': -0.35021573021298363,
 'compression_ratio': 1.3333333333333333,
 'no_speech_prob': 0.17020872235298157}